# Case study

In the case study we demonstrate how to generate counterfactual explanations by using our library on Statlog (German Credit Data) Data Set from UCI ML repository.  

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
np.random.seed(44)
# pd.set_option("display.max_rows", None)

2022-01-21 15:10:25.013256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-01-21 15:10:25.013288: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Data and model loading
We load statlog data which is wrapped in GermanData class. Additionally, we load a pretrained keras model which will be used for prediction making.

In [2]:
from data import GermanData

german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')

2022-01-21 15:10:28.063521: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-01-21 15:10:28.063548: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-21 15:10:28.063569: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (host-143-214): /proc/driver/nvidia/version does not exist
2022-01-21 15:10:28.063790: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
german_data.input.sample(5)

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
99.0,20.0,7057.0,3.0,4.0,36.0,2.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
488.0,10.0,1418.0,3.0,2.0,35.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
206.0,12.0,1935.0,4.0,4.0,43.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
939.0,24.0,6842.0,2.0,4.0,46.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
729.0,24.0,1275.0,2.0,4.0,36.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


### Test sample selection

We select one instance, for which we want to calculate the counterfactual. 

In [4]:
X_test = german_data.input.iloc[0]
X_test

duration                    6.0
credit                   1169.0
installment_percent         4.0
residence_duration          4.0
age                        67.0
                          ...  
employment_1..4 years       0.0
employment_4..7 years       0.0
employment_< 1 year         0.0
employment_>= 7 years       1.0
employment_unemployed       0.0
Name: 0.0, Length: 61, dtype: float64

### Data scaling

We need to scale the data for the model. The scale function uses StandardScaler from sklearn.

In [5]:
X_test_scaled = german_data.scale(X_test)
X_test_scaled

duration                -1.242314
credit                  -0.753586
installment_percent      0.936622
residence_duration       1.024474
age                      2.796337
                           ...   
employment_1..4 years   -0.707770
employment_4..7 years   -0.456573
employment_< 1 year     -0.454573
employment_>= 7 years    1.676163
employment_unemployed   -0.252646
Length: 61, dtype: float64

Now, we evaluate the model on the test sample.

In [6]:
model.predict(np.expand_dims(X_test_scaled, axis=0))

array([[0.00789263, 0.99210733]], dtype=float32)

The class predicted for this istance is 1, meaning bad customer score.

### Counterfactual explanations generation

Here we demonstrate how to generate counterfactual explanations using CADEX and FIMAP methods implemented in our library.

#### FIMAP


In [7]:
from counterfactuals.explainers import Fimap

model_predictions = model.predict(german_data.X_train)
model_predictions = np.argmax(model_predictions, axis=1)
fimap = Fimap()
fimap.fit(german_data.X_train, model_predictions)

cf_fimap = fimap.generate(X_test)


Training s

Training g


In [8]:
model.predict(german_data.scale(cf_fimap))

array([[0.69022167, 0.30977833]], dtype=float32)

The class predicted for the counterfactual is 0, meaning good credit score.

In [9]:
from counterfactuals.visualization import show

show(X_test, cf_fimap)

,X,X',change
duration,6.0,8.003710,2.003710
credit,1169.0,1168.619751,-0.380249
installment_percent,4.0,3.657258,-0.342742
residence_duration,4.0,5.199236,1.199236
age,67.0,65.779129,-1.220871
...,...,...,...
employment_1..4 years,0.0,1.258569,1.258569
employment_4..7 years,0.0,1.707869,1.707869
employment_< 1 year,0.0,-0.184148,-0.184148
employment_>= 7 years,1.0,1.134771,0.134771


We can see two things wrong with the generated counterfactual: 
- the categorical variables, originally represented in one-hot encoding, where changed to values different from 0 or 1
- the value of age variable decreased, resulting in poor quality of the counterfactual - we can't recommend someone to decrease their age in order to obtain credit

To fix it, we can use constraints, defined either in code or in spreadsheets (which can be used by users not familiar with programming).

In [10]:
from counterfactuals.constraints import OneHot, ValueMonotonicity

constraints = [OneHot("account_status", 7, 10), 
               OneHot("credit_history", 11, 15),
               OneHot("purpose", 16, 25), 
               OneHot("savings", 26, 30), 
               OneHot("sex_status", 31, 34),
               OneHot("debtors", 35, 37), 
               OneHot("property", 38, 41),
               OneHot("other_installment_plans", 42, 44), 
               OneHot("housing", 45, 47), 
               OneHot("job", 48, 51),
               OneHot("phone", 52, 53), 
               OneHot("foreign", 54, 55), 
               OneHot("employment", 56, 60),
               ValueMonotonicity(['age'], "increasing")
              ]

In [11]:
fimap = Fimap(constraints=constraints)
fimap.fit(german_data.X_train, model_predictions)

cf_fimap_constraints = fimap.generate(X_test)
cf_fimap_constraints


Training s

Training g


,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,6.052353,1168.973511,3.772412,3.497916,67.0,2.284397,0.323103,0.622258,0.891298,0.952552,...,1.328458,0.416872,0.224912,1.307038,0.111592,-0.569239,0.60002,1.452234,0.698143,0.672475


In [12]:
model.predict(german_data.scale(cf_fimap_constraints))

array([[0.01352705, 0.98647296]], dtype=float32)

In [13]:
show(X_test, cf_fimap_constraints)

,X,X',change
duration,6.0,6.052353,0.052353
credit,1169.0,1168.973511,-0.026489
installment_percent,4.0,3.772412,-0.227588
residence_duration,4.0,3.497916,-0.502084
existing_credits,2.0,2.284397,0.284397
people_maintained,1.0,0.323103,-0.676897
account_status_0..200 DM,0.0,0.622258,0.622258
account_status_< 0 DM,1.0,0.891298,-0.108702
account_status_>= 200 DM,0.0,0.952552,0.952552
account_status_no checking account,0.0,-1.539094,-1.539094


#### CADEX

For CADEX, we can either pass scaled instance and then unscale the obtained counterfactual or pass transform and inverse_transform parameters to the constructor.

In [14]:
from counterfactuals.explainers import Cadex

cadex = Cadex(model)
cf = cadex.generate(X_test_scaled)

In [15]:
model.predict(cf)

array([[0.50213647, 0.49786362]], dtype=float32)

In [16]:
from counterfactuals.visualization import show

show(X_test_scaled, cf)

,X,X',change
duration,-1.242314,0.814855,2.057169
account_status_< 0 DM,1.608553,3.665722,2.057169
account_status_no checking account,-0.780806,-2.837975,-2.057169
debtors_guarantor,-0.235391,-2.292560,-2.057169
foreign_yes,0.200779,2.257948,2.057169


In [17]:
show(X_test, german_data.unscale(cf))

,X,X',change
duration,6.0,3.031093e+01,2.431093e+01
account_status_< 0 DM,1.0,1.922402e+00,9.224022e-01
account_status_no checking account,0.0,-9.978828e-01,-9.978828e-01
purpose_car (new),0.0,-2.775558e-17,-2.775558e-17
savings_100..500 DM,0.0,1.387779e-17,1.387779e-17
debtors_guarantor,0.0,-4.588167e-01,-4.588167e-01
property_building society savings agreement,0.0,-2.775558e-17,-2.775558e-17
foreign_yes,1.0,1.397031e+00,3.970306e-01
employment_unemployed,0.0,6.938894e-18,6.938894e-18


In [18]:
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(X_test)

In [19]:
model.predict(german_data.scale(cf))

array([[0.50187844, 0.49812156]], dtype=float32)

In [20]:
show(X_test, cf)

,X,X',change
duration,6.0,3.030575e+01,2.430575e+01
account_status_< 0 DM,1.0,1.922206e+00,9.222056e-01
account_status_no checking account,0.0,-9.976701e-01,-9.976701e-01
purpose_car (new),0.0,-2.775558e-17,-2.775558e-17
savings_100..500 DM,0.0,1.387779e-17,1.387779e-17
debtors_guarantor,0.0,-4.587189e-01,-4.587189e-01
property_building society savings agreement,0.0,-2.775558e-17,-2.775558e-17
foreign_yes,1.0,1.396946e+00,3.969460e-01
employment_unemployed,0.0,6.938894e-18,6.938894e-18


In [21]:
show(X_test_scaled, german_data.scale(cf))

,X,X',change
duration,-1.242314,0.814417,2.056731
account_status_< 0 DM,1.608553,3.665284,2.056731
account_status_no checking account,-0.780806,-2.837537,-2.056731
debtors_guarantor,-0.235391,-2.292122,-2.056731
foreign_yes,0.200779,2.257509,2.056730


With constraints:

In [22]:
cadex = Cadex(model, n_changed=10, transform=german_data.scale, inverse_transform=german_data.unscale, constraints=constraints)
cf = cadex.generate(X_test)

In [23]:
model.predict(german_data.scale(cf))

array([[0.5000543 , 0.49994576]], dtype=float32)

In [24]:
show(X_test, cf)

,X,X',change
duration,6.0,2.388973e+01,1.788973e+01
credit,1169.0,5.297177e+03,4.128177e+03
age,67.0,4.994190e+01,-1.705810e+01
purpose_car (new),0.0,-2.775558e-17,-2.775558e-17
savings_100..500 DM,0.0,1.387779e-17,1.387779e-17
savings_< 100 DM,0.0,1.000000e+00,1.000000e+00
savings_unknown/ no savings account,1.0,0.000000e+00,-1.000000e+00
property_building society savings agreement,0.0,-2.775558e-17,-2.775558e-17
other_installment_plans_bank,0.0,1.000000e+00,1.000000e+00
other_installment_plans_none,1.0,0.000000e+00,-1.000000e+00


In [25]:
show(X_test_scaled, german_data.scale(cf))

,X,X',change
duration,-1.242314,0.271499,1.513813
credit,-0.753586,0.760225,1.513811
age,2.796337,1.282525,-1.513811
savings_< 100 DM,-1.237597,0.808018,2.045614
savings_unknown/ no savings account,2.209605,-0.452570,-2.662174
other_installment_plans_bank,-0.418023,2.392214,2.810237
other_installment_plans_none,0.492175,-2.031798,-2.523973
housing_own,0.656603,-1.522991,-2.179594
housing_rent,-0.490214,2.039924,2.530138


### ECE 

In [26]:
from counterfactuals.explainers import ECE

In [30]:
fimap = Fimap()
fimap.fit(german_data.X_train, model_predictions)

cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)


ece = ECE(3, columns=list(german_data.X_train.columns), bces=[cadex, fimap], dist=2, h=2, lambda_=0.2, n_jobs=1)
cfs = ece.generate(X_test)


Training s

Training g
CFS: [[ 5.01210451e+00  1.16846472e+03  3.44482589e+00  3.33146429e+00
   6.66258698e+01  1.27214313e-01  1.93031502e+00  1.26263690e+00
   1.20488560e+00  1.17338829e-01  5.39376855e-01 -2.17245281e-01
   1.97231412e+00  3.10641617e-01  3.32272679e-01 -1.08911979e+00
   8.03945363e-01 -4.09251064e-01  3.98719907e-01 -7.57368982e-01
   1.08544849e-01 -7.60281235e-02 -1.01671946e+00  1.22939336e+00
  -1.84509203e-01 -5.85987791e-03 -5.60930252e-01 -1.05429173e+00
   1.41240013e+00 -1.04228504e-01  6.92524314e-01  9.77810740e-01
  -3.81127834e-01 -2.88649172e-01  2.34464693e+00  1.24856985e+00
  -1.83286905e+00 -2.89122581e-01 -1.75949585e+00 -1.94741035e+00
  -4.26236749e-01  1.55276105e-01  4.03926134e-01  1.15471876e+00
   3.66095483e-01 -5.20429574e-02  9.39121842e-01  7.55374491e-01
   2.82501318e-02  7.38004982e-01  1.01168621e+00  9.26268160e-01
  -1.29049468e+00  5.87440670e-01 -9.07935500e-01  2.15428877e+00
   1.25136995e+00  1.97658017e-01 -1.26963031e+

/tmp/ipykernel_25842/3427338743.py:8: UserWarning: k parameter > number of aggregated counterfactuals. Changing k from 3 to 2
  cfs = ece.generate(X_test)
/tmp/ipykernel_25842/3427338743.py:8: UserWarning: knn's h parameter >= number of aggregated counterfactuals. Changing h from  2 to 1
  cfs = ece.generate(X_test)


In [31]:
cfs

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,30.328948,1169.0,4.0,4.0,67.0,2.0,1.0,0.0,1.923086,0.0,...,0.0,0.0,1.0,0.0,1.397325,0.0,0.0,0.0,1.0,6.938894e-18


In [29]:
from counterfactuals.visualization import compare
compare(X_test, cfs)

,X,CF1 change,constraint
number of attributes changed,0.0,12.0,
duration,6.0,24.325071,
credit,1169.0,-,
installment_percent,4.0,-,
residence_duration,4.0,-,
...,...,...,...
employment_1..4 years,0.0,-,
employment_4..7 years,0.0,-,
employment_< 1 year,0.0,-,
employment_>= 7 years,1.0,-,


For more information on the library, see our documentation